# Specification of default values of available data readers

> This module specifies the default values of the available data readers, their descriptions, and their widgets:

In [ ]:
#| default_exp readers/specs

In [ ]:
#| export

from abc import ABC, abstractmethod
from typing import List, Tuple, Optional, Dict, Any
from pathlib import PosixPath, Path

from findmycells.configs import DefaultConfigs, GUIConfigs

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export

class ReaderSpecsABC(ABC):
    
    @property
    @abstractmethod
    def reader_type(self) -> str:
        pass
    
    @property
    @abstractmethod
    def reader_widget_description(self) -> str:
        """
        Description that will be converted into HTML and displayed in the GUI
        right above the constructed widgets. Should therefore explain everything
        relevant for the user to understand the configuration options.
        """
        pass
    
    @property
    @abstractmethod
    def default_configs(self) -> DefaultConfigs:
        """
        Abstract method that requires its subclasses to define the `default_configs`
        as a property of the class. Thus, this will specify all configuration options
        that come with each subclass, while simultaneously also providing default values
        for each option and, moreover, defining what types of values are allowed for each
        option. Check out the implementation of `DefaultConfigs` in the configs module, or
        have a look at how this is implemented in one of the following ReaderSpecs 
        subclasses, like the `MicroscopyReaderSpecs` class.
        """
        pass    
    
    
    @property
    @abstractmethod
    def widget_names(self) -> Dict[str, str]:
        pass
    
    
    @property
    @abstractmethod
    def descriptions(self) -> Dict[str, str]:
        pass
    
    
    @property
    @abstractmethod
    def tooltips(self) -> Optional[Dict[str, str]]:
        return None    
    
    
    def initialize_gui_configs_and_widget(self) -> None:
        gui_configs = GUIConfigs(widget_names = self.widget_names,
                                 descriptions = self.descriptions,
                                 tooltips = self.tooltips)
        gui_configs.construct_widget(strategy_description = self.reader_widget_description,
                                     default_configs = self.default_configs)
        setattr(self, 'gui_configs', gui_configs)
        self.widget = self.gui_configs.strategy_widget
    
    
    def export_current_gui_config_values(self) -> Dict:
        return self.gui_configs.export_current_config_values()    

In [ ]:
#| export

class MicroscopyReaderSpecs(ReaderSpecsABC):
    
    @property
    def reader_type(self) -> str:
        return 'microscopy_images'
    
    @property
    def reader_widget_description(self) -> str:
        descripion = """
        The following widgets help you in loading your microscopy image data into findmycells, 
        while also aiming at limiting the required computing and memory resources. Therfore, 
        you have the choice to select precisely those color channels or plane indices, which 
        shall be analyzed, but also to load all channels & planes, if necessary.
        You also have the chance to test different settings until you found those that fit 
        your demands. For this, please specify your first combination of values and confirm them
        by clicking the "confirm microscopy reader settings" button. After also specifying ROI
        reader settings, simply head over to the "preprocess" page using the navigation buttons 
        in the top bar & preprocess a single file of your data. We recommend limiting the use of 
        preprocessing strategies for instance to the "convert to 8-bit" strategy, to get the best
        impression of what of your image data was loaded. You can inspect the preprocessed images
        in the "preprocessed_images" folder, which was created in your specified project root 
        directory. Feel free to head back to this page to refine the microscopy or ROI reader
        configurations & then preprocess the same file again to see the changes 
        (remember checking the "overwrite" checkbox when running the preprocessing). You can repeat
        this process until you found the right configurations for your project.
        """
        return description    
    
    @property
    def widget_names(self):
        widget_names = {'all_color_channels': 'Checkbox',
                        'specific_color_channel_idxs_range': 'IntRangeSlider',
                        'all_planes': 'Checkbox',
                        'specific_plane_idxs_range': 'IntRangeSlider',
                        'version_idx': 'BoundedIntText',
                        'tile_row_idx': 'BoundedIntText',
                        'tile_col_idx': 'BoundedIntText'}
        return widget_names

    @property
    def descriptions(self):
        descriptions = {'all_color_channels': 'load all color channels (if unchecked, you have to specify the range color channel below)',
                        'specific_color_channel_idxs_range': 'Range of color channel indices to load (only if "all color channels" is unchecked)',
                        'all_planes': 'load all image planes (if unchecked, you have to specify the range of planes below)',
                        'specific_plane_idxs_range': 'Range of plane indices to load (only if "all planes" is unchecked)',
                        'version_idx': '[Optional] Version index to load',
                        'tile_row_idx': '[Optional] Row index of tile to load',
                        'tile_col_idx': '[Optional] Column index of tile to load'}
        return descriptions
    
    @property
    def tooltips(self):
        return {}   
    
    
    @property
    def default_configs(self) -> DefaultConfigs:
        default_values = {'all_color_channels': True,
                          'specific_color_channel_idxs_range': (0, 1),
                          'all_planes': True,
                          'specific_plane_idxs_range': (0, 1),
                          'version_idx': 0,
                          'tile_row_idx': 0,
                          'tile_col_idx': 0}
        valid_types = {'all_color_channels': [bool],
                       'specific_color_channel_idxs_range': [tuple], # a tuple of integers
                       'all_planes': True,
                       'specific_plane_idxs_range': [tuple], # a tuple of integers
                       'version_idx': [int],
                       'tile_row_idx': [int],
                       'tile_col_idx': [int]}
        valid_ranges = {'specific_color_channel_idxs_range': (0, 3, 1), # are more possible? most implementations currently assume single color or RGB
                        'specific_plane_idxs_range': (0, 100, 1), # more should usually not be required?
                        'version_idx': (0, 999, 1), # 999 just to put an upper limit
                        'tile_row_idx': (0, 999, 1), # 999 just to put an upper limit
                        'tile_col_idx': (0, 999, 1)} # 999 just to put an upper limit        
        default_configs = DefaultConfigs(default_values = default_values, valid_types = valid_types, valid_value_ranges = valid_ranges)
        return default_configs

In [ ]:
#| export

class ROIReaderSpecs(ReaderSpecsABC):
    
    @property
    def reader_type(self) -> str:
        return 'rois'
    
    @property
    def reader_widget_description(self) -> str:
        descripion = """
        The following widgets allow you to specify where findmycells shall analyze
        your images. You can either provide files that denote the areas of each image
        that shall be analyzed & quantified (from here on referred to as ROI-files),
        or simply chose to analyze the entire image. Findmycells also supports the
        analysis of multiple ROIs per image. To ensure that the quantifications are
        done correctly, we recommend that you specify the corresponding ID of each ROI
        using the tool you used to create the ROI-files (e.g. Fiji). Findmycells will
        then match the quantification results based on the provided ROI IDs. For instance,
        if you have a microscopy image that shows the entire hippocampus, you may provide
        ROI files that contain ROIs with the IDs "DG", "CA3", and "CA1". Findmycells will
        then run the quantifications separately for each region and, importantly, not every
        image needs to contain all three ROIs.
        """
        return description
    
    @property
    def widget_names(self):
        widget_names = {'whole_image_as_roi': 'Checkbox',
                        'load_roi_ids_from_file': 'Checkbox'}
        return widget_names

    @property
    def descriptions(self):
        descriptions = {'whole_image_as_roi': 'load provided ROI files (= checked) or analyze whole images (= unchecked)',
                        'load_roi_ids_from_file': 'load (= checked) or ingore (= unchecked) ROI IDs from file'}
        return descriptions
    
    @property
    def tooltips(self):
        return {}   
    
    
    @property
    def default_configs(self) -> DefaultConfigs:
        default_values = {'whole_image_as_roi': False,
                          'load_roi_ids_from_file': True}
        valid_types = {'whole_image_as_roi': [bool],
                       'load_roi_ids_from_file': [bool]}     
        default_configs = DefaultConfigs(default_values = default_values, valid_types = valid_types)
        return default_configs

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()